# EVALUATION: Behavioral Correctness with LMEM

In [1]:
import json

import pandas as pd
from matplotlib import pyplot as plt
from utils.utils import *

%run utils/LMEM_helpers.ipynb

C:\Users\maure\AppData\Roaming\Python\Python311\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


Lade nötiges Paket: Matrix



Attache Paket: 'lmerTest'

Das folgende Objekt ist maskiert 'package:lme4':

    lmer

Das folgende Objekt ist maskiert 'package:stats':

    step



ciTools version 0.6.1 (C) Institute for Defense Analyses



Attache Paket: 'nlme'

Das folgende Objekt ist maskiert 'package:lme4':

    lmList

In addition: Warning message:
Paket 'nlme' wurde unter R Version 4.3.3 erstellt 


Lade nötiges Paket: mgcv
This is mgcv 1.9-0. For overview type 'help("mgcv-package")'.
Lade nötiges Paket: plotfunctions
Loaded package itsadug 2.4 (see 'help("itsadug")' ).
In addition: Warning messages:
1: Paket 'itsadug' wurde unter R Version 4.3.3 erstellt 
2: Paket 'plotfunctions' wurde unter R Version 4.3.3 erstellt 


### Load behavioral data

In [2]:
# Snippet data
behavioral_data = pd.read_csv(get_behavioral_data_path(
    final_data_exclusion=True, analysis_prep=True), sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})
display(behavioral_data)
print(behavioral_data.dtypes.to_dict())

,Snippet,Fixation_Start,End,Start,Duration,Answer,Correctness,Rating,right_answer,possible_wrong_answers,...,HDF_index,Participant,start_event,Block,Block_Index,Stimulus_id,Trial_No,Block_No,In_Block_No,Duration_log
0,16-obf-v0,130.144131,165.194542,135.137310,30.057232,3,0,1,7,5;3,...,0.0,P003,12,1,1,Stimulus/S 12,1,1,1,3.403103
1,5-clean-v0,174.369258,203.895033,179.370753,24.524280,false,0,1,true,false,...,0.0,P003,11,1,2,Stimulus/S 11,2,1,2,3.199664
2,61-clean-v0,211.250803,221.093158,216.250725,4.842433,4,1,1,4,2;0,...,0.0,P003,11,1,3,Stimulus/S 11,3,1,3,1.577417
3,13-clean-v0,228.432896,237.992304,233.416647,4.575657,5,1,1,5,6;3,...,0.0,P003,11,1,4,Stimulus/S 11,4,1,4,1.520750
4,10-obf-v0,245.032454,264.358911,250.001624,14.357288,1,1,0,1,0;2,...,0.0,P003,12,1,5,Stimulus/S 12,5,1,5,2.664258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,18-clean-v2,650.097955,671.641398,655.097237,16.544161,6,1,1,6,4;5,...,7.0,P106,11,3,20,Stimulus/S 11,68,3,20,2.806033
1723,54-obf-v2,678.104435,687.740039,683.096211,4.643827,6,0,1,0,6;2,...,7.0,P106,12,3,21,Stimulus/S 12,69,3,21,1.535539
1724,9-clean-v2,694.153446,707.572308,699.128912,8.443396,false,1,1,false,true,...,7.0,P106,11,3,22,Stimulus/S 11,70,3,22,2.133385
1725,12-obf-v2,713.678701,726.539764,718.661568,7.878196,4,1,1,4,3;2,...,7.0,P106,12,3,23,Stimulus/S 12,71,3,23,2.064099


{'Snippet': dtype('O'), 'Fixation_Start': dtype('float64'), 'End': dtype('float64'), 'Start': dtype('float64'), 'Duration': dtype('float64'), 'Answer': dtype('O'), 'Correctness': dtype('int64'), 'Rating': dtype('int64'), 'right_answer': dtype('O'), 'possible_wrong_answers': dtype('O'), 'number': dtype('O'), 'variant': dtype('O'), 'version': dtype('O'), 'Langhout AoC_category': dtype('O'), 'AoC_category': dtype('O'), 'Condition': dtype('O'), 'HDF_index': dtype('float64'), 'Participant': dtype('O'), 'start_event': dtype('int64'), 'Block': dtype('int64'), 'Block_Index': dtype('int64'), 'Stimulus_id': dtype('O'), 'Trial_No': dtype('int64'), 'Block_No': dtype('int64'), 'In_Block_No': dtype('int64'), 'Duration_log': dtype('float64')}


In [3]:
# Correctness: Binomial on normal values (0/1)
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion base model binomial')
# best model according to AIC (see below for more complex models, reduction to this model performed)

Correctness ~ 1 + Condition + (1 + Condition | Participant) + (1 + Condition | number)
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition + (1 + Condition | Participant) +  
    (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1215.6   1259.3   -599.8   1199.6     1719 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.3594  0.0856  0.1974  0.3407  3.8404 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0033   1.0017        
             Conditionconfusing 0.2941   0.5423   -0.65
 number      (Intercept)        2.2404   1.4968        
             Conditionconfusing 1.6341   1.2783   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)          3.0374     0.4198   7.235 4

In [4]:
# 3wise interaction failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no in_block_no 3wise binomial')

Correctness ~ 1 + Condition * Block_No * In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.15758 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition * Block_No * In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1223.6   1299.9   -597.8   1195.6     1713 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.0012  0.0823  0.1982  0.3398  3.7563 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0317   1.0157        
             Conditionconfusing 0.3217   0.5672   -0.66
 number      (Intercept)        2.2643   1.5048        
             Conditionconfusing 1.6550   1.2865   0.10 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                                        Estimate Std. Error z value Pr(>|z|)   
(Intercept)                              2.10742    0.79147   2.663  0.00775 **
Condition

In [5]:
# all 2wise interactions failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No + {CONDITION} * In_Block_No + Block_No * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no in_block_no all 2wise binomial')

Correctness ~ 1 + Condition * Block_No + Condition * In_Block_No + Block_No * In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00207913 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition * Block_No + Condition * In_Block_No +  
    Block_No * In_Block_No + (1 + Condition | Participant) +  
    (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1222.4   1293.3   -598.2   1196.4     1714 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.3868  0.0825  0.1991  0.3410  3.7400 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0213   1.0106        
             Conditionconfusing 0.3102   0.5569   -0.66
 number      (Intercept)        2.2622   1.5041        
             Conditionconfusing 1.6283   1.2761   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     2.499841   0.656525   

In [6]:
# all models with 2/3 2wise interactions failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No + {CONDITION} * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no in_block_no 2 2wise with c binomial')
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No + Block_No * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no in_block_no 2 2wise with b binomial')
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * In_Block_No + Block_No * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no in_block_no 2 2wise with i binomial')

Correctness ~ 1 + Condition * Block_No + Condition * In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00632755 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition * Block_No + Condition * In_Block_No +  
    (1 + Condition | Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1220.5   1286.0   -598.3   1196.5     1715 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.5243  0.0827  0.1982  0.3421  3.7546 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0153   1.0076        
             Conditionconfusing 0.3075   0.5545   -0.65
 number      (Intercept)        2.2603   1.5034        
             Conditionconfusing 1.6367   1.2793   0.10 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     2.630739   0.562837   4.674 2.95e-06 ***
Conditioncon

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.0172112 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition * Block_No + Block_No * In_Block_No +  
    (1 + Condition | Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1220.7   1286.2   -598.4   1196.7     1715 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.1824  0.0825  0.1982  0.3422  3.7897 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0214   1.0106        
             Conditionconfusing 0.3084   0.5553   -0.66
 number      (Intercept)        2.2707   1.5069        
             Conditionconfusing 1.6267   1.2754   0.10 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  2.400178   0.631437   3.801 0.000144 ***
Conditionconfusing 

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.0161127 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition * In_Block_No + Block_No * In_Block_No +  
    (1 + Condition | Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1222.8   1288.3   -599.4   1198.8     1715 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.1172  0.0852  0.1986  0.3379  3.7538 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0051   1.0026        
             Conditionconfusing 0.3009   0.5486   -0.65
 number      (Intercept)        2.2346   1.4949        
             Conditionconfusing 1.6283   1.2760   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     2.899620   0.617508   4.696 2.66e-06 ***
Conditionc

In [7]:
# 2/3 models with 1 2wise interaction failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + Block_No * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no i in_block_no 2wise binomial')
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + Block_No + {CONDITION} * In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion i in_block_no block_no 2wise binomial')

Correctness ~ 1 + Condition + Block_No * In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.0571756 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition + Block_No * In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1221.1   1281.1   -599.6   1199.1     1716 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.8934  0.0851  0.1989  0.3408  3.7978 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0069   1.0034        
             Conditionconfusing 0.3021   0.5497   -0.65
 number      (Intercept)        2.2450   1.4983        
             Conditionconfusing 1.6255   1.2749   0.12 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)           2.792751   0.585681   4.768 1.86e-06 ***
Conditionconfusing   -1.565759   0.369435  

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00979904 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Block_No + Condition * In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1220.9   1280.9   -599.5   1198.9     1716 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.2378  0.0858  0.1988  0.3381  3.7619 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0063   1.0031        
             Conditionconfusing 0.3006   0.5483   -0.65
 number      (Intercept)        2.2354   1.4951        
             Conditionconfusing 1.6341   1.2783   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     2.990009   0.521005   5.739 9.53e-09 ***
Block_No               

In [8]:
# 1 models with 1 2wise interaction converged Correctness ~ 1 + Condition + Block_No * In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No + In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion i block_no in_block_no 2wise binomial')

Correctness ~ 1 + Condition * Block_No + In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition * Block_No + In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1218.8   1278.8   -598.4   1196.8     1716 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.3135  0.0829  0.1980  0.3422  3.7992 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0184   1.0092        
             Conditionconfusing 0.3068   0.5539   -0.66
 number      (Intercept)        2.2699   1.5066        
             Conditionconfusing 1.6343   1.2784   0.10 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                             Estimate Std. Error z value 

In [9]:
# reduction round 1 test 1: remove interaction failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + Block_No + In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion in_block_no block_no 1wise binomial')

Correctness ~ 1 + Condition + Block_No + In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00381907 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition + Block_No + In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1219.2   1273.7   -599.6   1199.2     1717 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.0047  0.0850  0.1988  0.3397  3.8059 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0031   1.0015        
             Conditionconfusing 0.2978   0.5457   -0.65
 number      (Intercept)        2.2412   1.4971        
             Conditionconfusing 1.6335   1.2781   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)         2.879897   0.477696   6.029 1.65e-09 ***
Conditionconfusing -1.568187   0.369574  -4.243

In [10]:
# reduction round 1 test 2: remove factor in_block_no failed to converge
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} * Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion i block_no 2wise binomial')

Correctness ~ 1 + Condition * Block_No + (1 + Condition | Participant) + (1 + Condition | number)


In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.00324067 (tol = 0.002, component 1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition * Block_No + (1 + Condition | Participant) +  
    (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1217.0   1271.5   -598.5   1197.0     1717 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.4351  0.0833  0.1986  0.3411  3.8418 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0181   1.0090        
             Conditionconfusing 0.3018   0.5493   -0.66
 number      (Intercept)        2.2689   1.5063        
             Conditionconfusing 1.6367   1.2793   0.10 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                   2.5754     0.5055   5.095 3.49e-07 ***
Conditionconfusing           -0.9922     0.52

In [11]:
# further reduction only 2 factors condition block_no
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion block_no 1wise binomial')

Correctness ~ 1 + Condition + Block_No + (1 + Condition | Participant) + (1 + Condition | number)
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Correctness ~ 1 + Condition + Block_No + (1 + Condition | Participant) +  
    (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1217.3   1266.4   -599.6   1199.3     1718 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.1091  0.0852  0.1988  0.3401  3.8423 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0046   1.002         
             Conditionconfusing 0.2948   0.543    -0.65
 number      (Intercept)        2.2408   1.497         
             Conditionconfusing 1.6353   1.279    0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)         2.926

In [12]:
# further reduction only 2 factors condition in_block_no
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + In_Block_No + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion in_block_no 1wise binomial')
# best model compared to below

Correctness ~ 1 + Condition + In_Block_No + (1 + Condition | Participant) + (1 + Condition | number)
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition + In_Block_No + (1 + Condition |  
    Participant) + (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1217.5   1266.6   -599.8   1199.5     1718 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.2383  0.0851  0.1984  0.3404  3.8010 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0026   1.0013        
             Conditionconfusing 0.2977   0.5456   -0.65
 number      (Intercept)        2.2421   1.4974        
             Conditionconfusing 1.6315   1.2773   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        

In [13]:
# ultimate reduction, converges
res2 = create_glmer_in_R(
    f'{BEHAVIORAL_COLUMN_CORRECTNESS} ~ 1 + {CONDITION} + ' +
    f'(1 + {CONDITION} | {PARTICIPANT}) + ' +
    f'(1 + {CONDITION} | {SNIPPET_COLUMN_NUMBER})',
    behavioral_data, LMEM_FAMILY_BINOMIAL, description='correctness confusion binomial final')

Correctness ~ 1 + Condition + (1 + Condition | Participant) + (1 + Condition | number)
Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Correctness ~ 1 + Condition + (1 + Condition | Participant) +  
    (1 + Condition | number)
   Data: data

     AIC      BIC   logLik deviance df.resid 
  1215.6   1259.3   -599.8   1199.6     1719 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.3594  0.0856  0.1974  0.3407  3.8404 

Random effects:
 Groups      Name               Variance Std.Dev. Corr 
 Participant (Intercept)        1.0033   1.0017        
             Conditionconfusing 0.2941   0.5423   -0.65
 number      (Intercept)        2.2404   1.4968        
             Conditionconfusing 1.6341   1.2783   0.11 
Number of obs: 1727, groups:  Participant, 24; number, 24

Fixed effects:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)          3.0374     0.4198   7.235 4